In [40]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
cd ..

C:\Projects\python\recommender


In [2]:
from functools import partial
from pathlib import Path

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

from utils import get_log_dir

In [3]:
from datasets import SeqTopcoder
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM
from models.fm_learner import simple_loss, trans_loss

In [7]:
DEVICE = T.cuda.current_device()
BATCH = 2000
SHUFFLE = True
WORKERS = 0
NEG_SAMPLE = 5
REGS_PATH = Path("./inputs/topcoder/regs.csv")
CHAG_PATH = Path("./inputs/topcoder/challenge.csv")

In [9]:
db = SeqTopcoder(regs_path=REGS_PATH, chag_path=CHAG_PATH)
db

Read dataset in inputs\topcoder\regs.csv
Original regs shape: (610025, 3)
Original registants size: 60017
Original challenges size: 39916
Filter dataframe shape: (544568, 3)
Index(['challengeId', 'period', 'date', 'prizes', 'technologies', 'platforms'], dtype='object')


In [10]:
db.config_db(batch_size=BATCH,
             shuffle=SHUFFLE,
             num_workers=WORKERS,
             device=DEVICE,
             neg_sample=NEG_SAMPLE)

In [11]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

### Create criterion

In [12]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1
TRANS_REG = 1

In [13]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x00000299882EC7B8>, 1, 1)

In [14]:
trans_loss_callback = partial(trans_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_loss_callback

functools.partial(<function trans_loss at 0x00000299882CB950>, 1, 1, 1)

### Train model

#### Hyper-parameter

In [15]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

#### Train FM Model

In [16]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [17]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
fm_model

TorchFM()

In [18]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [19]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, db)
fm_learner

In [20]:
fm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [21]:
fm_learner.fit(epoch=5,
               log_dir=get_log_dir('seq_topcoder', 'fm'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 38491.63848501149
Epoch 0 step 1: training accuarcy: 0.322
Epoch 0 step 1: training loss: 37296.29721029236
Epoch 0 step 2: training accuarcy: 0.3655
Epoch 0 step 2: training loss: 36060.13888788689
Epoch 0 step 3: training accuarcy: 0.4345
Epoch 0 step 3: training loss: 34891.4932175206
Epoch 0 step 4: training accuarcy: 0.5075000000000001
Epoch 0 step 4: training loss: 33787.61291161411
Epoch 0 step 5: training accuarcy: 0.5615
Epoch 0 step 5: training loss: 32763.335192424827
Epoch 0 step 6: training accuarcy: 0.6035
Epoch 0 step 6: training loss: 31799.103568248593
Epoch 0 step 7: training accuarcy: 0.6445
Epoch 0 step 7: training loss: 30902.477741931438
Epoch 0 step 8: training accuarcy: 0.655
Epoch 0 step 8: training loss: 29939.08243198299
Epoch 0 step 9: training accuarcy: 0.6995
Epoch 0 step 9: training loss: 29042.39969591284
Epoch 0 step 10: training accuarcy: 0.735
Epoch 0 step 10: training loss: 28232.15831329322
Epoch 0 step 11: tr

Epoch 0 step 86: training loss: 2583.2870419622363
Epoch 0 step 87: training accuarcy: 0.9675
Epoch 0 step 87: training loss: 2510.1658766706582
Epoch 0 step 88: training accuarcy: 0.966
Epoch 0 step 88: training loss: 2429.3595244899984
Epoch 0 step 89: training accuarcy: 0.967
Epoch 0 step 89: training loss: 2335.163170841124
Epoch 0 step 90: training accuarcy: 0.972
Epoch 0 step 90: training loss: 2266.4117187967418
Epoch 0 step 91: training accuarcy: 0.9705
Epoch 0 step 91: training loss: 2210.5986547968396
Epoch 0 step 92: training accuarcy: 0.9655
Epoch 0 step 92: training loss: 2135.9960769289733
Epoch 0 step 93: training accuarcy: 0.964
Epoch 0 step 93: training loss: 2064.2956218820577
Epoch 0 step 94: training accuarcy: 0.971
Epoch 0 step 94: training loss: 2013.3649624560735
Epoch 0 step 95: training accuarcy: 0.968
Epoch 0 step 95: training loss: 1942.529633240988
Epoch 0 step 96: training accuarcy: 0.968
Epoch 0 step 96: training loss: 1877.0055457768146
Epoch 0 step 97: t

Epoch 0 step 173: training accuarcy: 0.9805
Epoch 0 step 173: training loss: 303.8813086886423
Epoch 0 step 174: training accuarcy: 0.992
Epoch 0 step 174: training loss: 310.3144314127424
Epoch 0 step 175: training accuarcy: 0.986
Epoch 0 step 175: training loss: 301.6742089193331
Epoch 0 step 176: training accuarcy: 0.986
Epoch 0 step 176: training loss: 289.32933664338225
Epoch 0 step 177: training accuarcy: 0.9875
Epoch 0 step 177: training loss: 289.3618106355673
Epoch 0 step 178: training accuarcy: 0.9865
Epoch 0 step 178: training loss: 279.4964899817554
Epoch 0 step 179: training accuarcy: 0.989
Epoch 0 step 179: training loss: 289.20747936951943
Epoch 0 step 180: training accuarcy: 0.9855
Epoch 0 step 180: training loss: 278.86956698372063
Epoch 0 step 181: training accuarcy: 0.9855
Epoch 0 step 181: training loss: 276.4663721659801
Epoch 0 step 182: training accuarcy: 0.9915
Epoch 0 step 182: training loss: 272.2153539873596
Epoch 0 step 183: training accuarcy: 0.984
Epoch 0 

Epoch 0 step 259: training accuarcy: 0.9915
Epoch 0 step 259: training loss: 168.1945615915896
Epoch 0 step 260: training accuarcy: 0.9935
Epoch 0 step 260: training loss: 163.89993818122878
Epoch 0 step 261: training accuarcy: 0.994
Epoch 0 step 261: training loss: 157.5699326275507
Epoch 0 step 262: training accuarcy: 0.996
Epoch 0 step 262: training loss: 122.55542614316795
Epoch 0 step 263: training accuarcy: 0.9897435897435898
Epoch 0: train loss 4893.037013669688, train accuarcy 0.9411860704421997
Epoch 0: valid loss 605.7904897973084, valid accuarcy 0.9905195236206055


...

Epoch: 4
Epoch 4 step 1052: training loss: 72.72105277164918
Epoch 4 step 1053: training accuarcy: 0.999
Epoch 4 step 1053: training loss: 77.93215464233515
Epoch 4 step 1054: training accuarcy: 0.999
Epoch 4 step 1054: training loss: 77.76225047042777
Epoch 4 step 1055: training accuarcy: 0.9985
Epoch 4 step 1055: training loss: 77.32250261089878
Epoch 4 step 1056: training accuarcy: 0.9965
Epoch 4 step 1056: training loss: 83.90414791490176
Epoch 4 step 1057: training accuarcy: 0.9975
Epoch 4 step 1057: training loss: 86.10148702151633
Epoch 4 step 1058: training accuarcy: 0.997
Epoch 4 step 1058: training loss: 82.44206099960529
Epoch 4 step 1059: training accuarcy: 0.9975
Epoch 4 step 1059: training loss: 78.1009582064336
Epoch 4 step 1060: training accuarcy: 0.9995
Epoch 4 step 1060: training loss: 73.19186944221843
Epoch 4 step 1061: training accuarcy: 0.999
Epoch 4 step 1061: training loss: 76.25496077408869
Epoch 4 step 1062: training accuarcy: 0.9985
Epoch 4 step 1062: trainin

Epoch 4 step 1137: training accuarcy: 0.998
Epoch 4 step 1137: training loss: 71.87503486655606
Epoch 4 step 1138: training accuarcy: 0.998
Epoch 4 step 1138: training loss: 78.7090981171257
Epoch 4 step 1139: training accuarcy: 0.998
Epoch 4 step 1139: training loss: 73.35889559097228
Epoch 4 step 1140: training accuarcy: 0.9995
Epoch 4 step 1140: training loss: 79.72226704577253
Epoch 4 step 1141: training accuarcy: 0.999
Epoch 4 step 1141: training loss: 79.29290496878298
Epoch 4 step 1142: training accuarcy: 0.998
Epoch 4 step 1142: training loss: 80.26239862485535
Epoch 4 step 1143: training accuarcy: 0.997
Epoch 4 step 1143: training loss: 74.29393184031116
Epoch 4 step 1144: training accuarcy: 0.9985
Epoch 4 step 1144: training loss: 73.01828405875892
Epoch 4 step 1145: training accuarcy: 0.998
Epoch 4 step 1145: training loss: 74.93337061686027
Epoch 4 step 1146: training accuarcy: 0.9975
Epoch 4 step 1146: training loss: 80.15301326594172
Epoch 4 step 1147: training accuarcy: 

Epoch 4 step 1222: training accuarcy: 0.9995
Epoch 4 step 1222: training loss: 74.92359162762563
Epoch 4 step 1223: training accuarcy: 0.998
Epoch 4 step 1223: training loss: 84.31135476574738
Epoch 4 step 1224: training accuarcy: 0.999
Epoch 4 step 1224: training loss: 79.9002021190812
Epoch 4 step 1225: training accuarcy: 0.9995
Epoch 4 step 1225: training loss: 70.33057597386022
Epoch 4 step 1226: training accuarcy: 0.998
Epoch 4 step 1226: training loss: 78.13646766789287
Epoch 4 step 1227: training accuarcy: 0.998
Epoch 4 step 1227: training loss: 78.85248120253226
Epoch 4 step 1228: training accuarcy: 0.998
Epoch 4 step 1228: training loss: 76.18430906708866
Epoch 4 step 1229: training accuarcy: 0.9985
Epoch 4 step 1229: training loss: 75.4696267718135
Epoch 4 step 1230: training accuarcy: 0.9985
Epoch 4 step 1230: training loss: 75.86598780391948
Epoch 4 step 1231: training accuarcy: 0.999
Epoch 4 step 1231: training loss: 86.69235213158778
Epoch 4 step 1232: training accuarcy: 

Epoch 4 step 1307: training accuarcy: 0.998
Epoch 4 step 1307: training loss: 71.39991421353481
Epoch 4 step 1308: training accuarcy: 0.9985
Epoch 4 step 1308: training loss: 82.12035057453046
Epoch 4 step 1309: training accuarcy: 1.0
Epoch 4 step 1309: training loss: 74.98332669669469
Epoch 4 step 1310: training accuarcy: 0.9975
Epoch 4 step 1310: training loss: 75.28519357555795
Epoch 4 step 1311: training accuarcy: 0.9995
Epoch 4 step 1311: training loss: 76.36441178879241
Epoch 4 step 1312: training accuarcy: 0.9975
Epoch 4 step 1312: training loss: 76.68488594109112
Epoch 4 step 1313: training accuarcy: 0.999
Epoch 4 step 1313: training loss: 75.4159289031391
Epoch 4 step 1314: training accuarcy: 0.9975
Epoch 4 step 1314: training loss: 57.42034484899322
Epoch 4 step 1315: training accuarcy: 0.9974358974358974
Epoch 4: train loss 77.51178102284467, train accuarcy 0.9949812889099121
Epoch 4: valid loss 352.3283909370639, valid accuarcy 0.9964019060134888


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [27:16<00:00, 327.43s/it]


In [26]:
del fm_model
T.cuda.empty_cache()

NameError: name 'fm_model' is not defined

### Train HRM FM Model

In [23]:
hrm_model = TorchHrmFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
hrm_model

TorchHrmFM()

In [24]:
adam_opt = optim.Adam(hrm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [25]:
hrm_learner = FMLearner(hrm_model, adam_opt, schedular, db)
hrm_learner

In [28]:
hrm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [ ]:
hrm_learner.fit(epoch=5,
                log_dir=get_log_dir('seq_topcoder', 'hrm'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 42819.92472716793
Epoch 0 step 1: training accuarcy: 0.5235
Epoch 0 step 1: training loss: 41781.00844523763
Epoch 0 step 2: training accuarcy: 0.5135
Epoch 0 step 2: training loss: 40377.03383102127
Epoch 0 step 3: training accuarcy: 0.5275
Epoch 0 step 3: training loss: 39087.03981484522
Epoch 0 step 4: training accuarcy: 0.5285
Epoch 0 step 4: training loss: 38546.408552017965
Epoch 0 step 5: training accuarcy: 0.514
Epoch 0 step 5: training loss: 37357.36722897061
Epoch 0 step 6: training accuarcy: 0.5195
Epoch 0 step 6: training loss: 35973.60338429364
Epoch 0 step 7: training accuarcy: 0.522
Epoch 0 step 7: training loss: 34682.32533080768
Epoch 0 step 8: training accuarcy: 0.5485
Epoch 0 step 8: training loss: 34184.93159070983
Epoch 0 step 9: training accuarcy: 0.526
Epoch 0 step 9: training loss: 33033.03608529885
Epoch 0 step 10: training accuarcy: 0.5425
Epoch 0 step 10: training loss: 31927.703055961145
Epoch 0 step 11: training accua

Epoch 0 step 87: training loss: 3231.609322376661
Epoch 0 step 88: training accuarcy: 0.5775
Epoch 0 step 88: training loss: 3177.266716641235
Epoch 0 step 89: training accuarcy: 0.5755
Epoch 0 step 89: training loss: 3096.6982001874267
Epoch 0 step 90: training accuarcy: 0.5745
Epoch 0 step 90: training loss: 3027.00633871001
Epoch 0 step 91: training accuarcy: 0.581
Epoch 0 step 91: training loss: 2989.0730920266324
Epoch 0 step 92: training accuarcy: 0.5505
Epoch 0 step 92: training loss: 2910.784283710388
Epoch 0 step 93: training accuarcy: 0.584
Epoch 0 step 93: training loss: 2863.0436492025647
Epoch 0 step 94: training accuarcy: 0.5700000000000001
Epoch 0 step 94: training loss: 2778.609765932709
Epoch 0 step 95: training accuarcy: 0.5945
Epoch 0 step 95: training loss: 2734.366319276499
Epoch 0 step 96: training accuarcy: 0.5925
Epoch 0 step 96: training loss: 2693.315033934541
Epoch 0 step 97: training accuarcy: 0.5690000000000001
Epoch 0 step 97: training loss: 2631.619137117

Epoch 0 step 173: training accuarcy: 0.622
Epoch 0 step 173: training loss: 1414.9737667272586
Epoch 0 step 174: training accuarcy: 0.6285000000000001
Epoch 0 step 174: training loss: 1429.33060624277
Epoch 0 step 175: training accuarcy: 0.606
Epoch 0 step 175: training loss: 1420.7935036779036
Epoch 0 step 176: training accuarcy: 0.6115
Epoch 0 step 176: training loss: 1415.4114227858433
Epoch 0 step 177: training accuarcy: 0.6135
Epoch 0 step 177: training loss: 1410.6567030656884
Epoch 0 step 178: training accuarcy: 0.625
Epoch 0 step 178: training loss: 1425.2908135379957
Epoch 0 step 179: training accuarcy: 0.593
Epoch 0 step 179: training loss: 1410.8200254771439
Epoch 0 step 180: training accuarcy: 0.6095
Epoch 0 step 180: training loss: 1412.2608280959723
Epoch 0 step 181: training accuarcy: 0.609
Epoch 0 step 181: training loss: 1401.468516024609
Epoch 0 step 182: training accuarcy: 0.6245
Epoch 0 step 182: training loss: 1403.8889403061905
Epoch 0 step 183: training accuarcy:

Epoch 0 step 258: training loss: 1356.371755687768
Epoch 0 step 259: training accuarcy: 0.6225
Epoch 0 step 259: training loss: 1361.9661363982937
Epoch 0 step 260: training accuarcy: 0.604
Epoch 0 step 260: training loss: 1357.778049080029
Epoch 0 step 261: training accuarcy: 0.6
Epoch 0 step 261: training loss: 1354.8313291723446
Epoch 0 step 262: training accuarcy: 0.6035
Epoch 0 step 262: training loss: 543.1271567781893
Epoch 0 step 263: training accuarcy: 0.5935897435897436
Epoch 0: train loss 6135.840041508194, train accuarcy 0.5847347378730774
Epoch 0: valid loss 6706.5613106402725, valid accuarcy 0.5839095115661621


...

Epoch 3 step 958: training loss: 1337.8067107111467
Epoch 3 step 959: training accuarcy: 0.6095
Epoch 3 step 959: training loss: 1337.3080971676152
Epoch 3 step 960: training accuarcy: 0.6145
Epoch 3 step 960: training loss: 1335.6716432501678
Epoch 3 step 961: training accuarcy: 0.6275000000000001
Epoch 3 step 961: training loss: 1338.4956425443247
Epoch 3 step 962: training accuarcy: 0.6045
Epoch 3 step 962: training loss: 1323.3131806669346
Epoch 3 step 963: training accuarcy: 0.6215
Epoch 3 step 963: training loss: 1342.5994801259912
Epoch 3 step 964: training accuarcy: 0.611
Epoch 3 step 964: training loss: 1328.644712559443
Epoch 3 step 965: training accuarcy: 0.6055
Epoch 3 step 965: training loss: 1339.3818996185348
Epoch 3 step 966: training accuarcy: 0.607
Epoch 3 step 966: training loss: 1328.3340417970899
Epoch 3 step 967: training accuarcy: 0.6045
Epoch 3 step 967: training loss: 1330.1169269208685
Epoch 3 step 968: training accuarcy: 0.622
Epoch 3 step 968: training loss:

Epoch 3 step 1043: training accuarcy: 0.6095
Epoch 3 step 1043: training loss: 1321.1676597863054
Epoch 3 step 1044: training accuarcy: 0.617
Epoch 3 step 1044: training loss: 1326.4585371900853
Epoch 3 step 1045: training accuarcy: 0.625
Epoch 3 step 1045: training loss: 1340.6061712193898
Epoch 3 step 1046: training accuarcy: 0.611
Epoch 3 step 1046: training loss: 1329.3108433416369
Epoch 3 step 1047: training accuarcy: 0.6245
Epoch 3 step 1047: training loss: 1319.4485375152597
Epoch 3 step 1048: training accuarcy: 0.6295000000000001
Epoch 3 step 1048: training loss: 1346.7429406975498
Epoch 3 step 1049: training accuarcy: 0.608
Epoch 3 step 1049: training loss: 1352.0318605308125
Epoch 3 step 1050: training accuarcy: 0.6185
Epoch 3 step 1050: training loss: 1338.3062320061883
Epoch 3 step 1051: training accuarcy: 0.616
Epoch 3 step 1051: training loss: 524.2369725879482
Epoch 3 step 1052: training accuarcy: 0.65
Epoch 3: train loss 1331.6161211012734, train accuarcy 0.614125192165

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                  | 4/5 [21:49<05:26, 326.91s/it]

Epoch: 4
Epoch 4 step 1052: training loss: 1330.4050943938764
Epoch 4 step 1053: training accuarcy: 0.614
Epoch 4 step 1053: training loss: 1321.4621151655
Epoch 4 step 1054: training accuarcy: 0.6325000000000001
Epoch 4 step 1054: training loss: 1333.322320605249
Epoch 4 step 1055: training accuarcy: 0.629
Epoch 4 step 1055: training loss: 1333.2582405160251
Epoch 4 step 1056: training accuarcy: 0.614
Epoch 4 step 1056: training loss: 1318.8609138760892
Epoch 4 step 1057: training accuarcy: 0.6315000000000001
Epoch 4 step 1057: training loss: 1337.4540537086648
Epoch 4 step 1058: training accuarcy: 0.617
Epoch 4 step 1058: training loss: 1327.9173864010597
Epoch 4 step 1059: training accuarcy: 0.6285000000000001
Epoch 4 step 1059: training loss: 1325.2039352832603
Epoch 4 step 1060: training accuarcy: 0.631
Epoch 4 step 1060: training loss: 1332.1277522854386
Epoch 4 step 1061: training accuarcy: 0.6125
Epoch 4 step 1061: training loss: 1348.116782975801
Epoch 4 step 1062: training ac

Epoch 4 step 1136: training accuarcy: 0.623
Epoch 4 step 1136: training loss: 1324.7643355956347
Epoch 4 step 1137: training accuarcy: 0.6195
Epoch 4 step 1137: training loss: 1344.1825062753378
Epoch 4 step 1138: training accuarcy: 0.5995
Epoch 4 step 1138: training loss: 1333.4270629858445
Epoch 4 step 1139: training accuarcy: 0.6215
Epoch 4 step 1139: training loss: 1332.8734374454455
Epoch 4 step 1140: training accuarcy: 0.6125
Epoch 4 step 1140: training loss: 1318.342133323241
Epoch 4 step 1141: training accuarcy: 0.6335000000000001


In [27]:
del hrm_model
T.cuda.empty_cache()

### Train PRME FM Model

In [28]:
prme_model = TorchPrmeFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
prme_model

TorchPrmeFM()

In [29]:
adam_opt = optim.Adam(prme_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [30]:
prme_learner = FMLearner(prme_model, adam_opt, schedular, db)
prme_learner

In [ ]:
prme_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [31]:
prme_learner.fit(epoch=5,
                 log_dir=get_log_dir('seq_topcoder', 'prme'))

  0%|                                                                                                                                                                 | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 37514.48137421203
Epoch 0 step 1: training accuarcy: 0.527
Epoch 0 step 1: training loss: 36473.747391143785
Epoch 0 step 2: training accuarcy: 0.5055000000000001
Epoch 0 step 2: training loss: 35454.346101311516
Epoch 0 step 3: training accuarcy: 0.5015000000000001
Epoch 0 step 3: training loss: 34454.33436229627
Epoch 0 step 4: training accuarcy: 0.484
Epoch 0 step 4: training loss: 33472.234940401846
Epoch 0 step 5: training accuarcy: 0.5105000000000001
Epoch 0 step 5: training loss: 32522.437981548668
Epoch 0 step 6: training accuarcy: 0.5055000000000001
Epoch 0 step 6: training loss: 31582.993115826408
Epoch 0 step 7: training accuarcy: 0.5135
Epoch 0 step 7: training loss: 30670.02176044439
Epoch 0 step 8: training accuarcy: 0.5155
Epoch 0 step 8: training loss: 29788.87466102316
Epoch 0 step 9: training accuarcy: 0.507
Epoch 0 step 9: training loss: 28914.7417543135
Epoch 0 step 10: training accuarcy: 0.5165
Epoch 0 step 10: training loss:

Epoch 0 step 171: training loss: 1403.2463424379605
Epoch 0 step 172: training accuarcy: 0.64
Epoch 0 step 172: training loss: 1402.6984465602586
Epoch 0 step 173: training accuarcy: 0.6355000000000001
Epoch 0 step 173: training loss: 1400.553143916913
Epoch 0 step 174: training accuarcy: 0.66
Epoch 0 step 174: training loss: 1400.4041205164006
Epoch 0 step 175: training accuarcy: 0.648
Epoch 0 step 175: training loss: 1397.8913055722246
Epoch 0 step 176: training accuarcy: 0.662
Epoch 0 step 176: training loss: 1397.7636800453208
Epoch 0 step 177: training accuarcy: 0.6465
Epoch 0 step 177: training loss: 1397.2701884359565
Epoch 0 step 178: training accuarcy: 0.655
Epoch 0 step 178: training loss: 1395.899843048871
Epoch 0 step 179: training accuarcy: 0.6735
Epoch 0 step 179: training loss: 1395.2441839108803
Epoch 0 step 180: training accuarcy: 0.6715
Epoch 0 step 180: training loss: 1395.9703598188728
Epoch 0 step 181: training accuarcy: 0.646
Epoch 0 step 181: training loss: 1394.

 12%|███████████████████                                                                                                                                     | 1/8 [01:56<13:35, 116.46s/it]

Epoch: 1
Epoch 1 step 263: training loss: 1381.9288631656204
Epoch 1 step 264: training accuarcy: 0.717
Epoch 1 step 264: training loss: 1382.8740386957263
Epoch 1 step 265: training accuarcy: 0.7205
Epoch 1 step 265: training loss: 1383.163731968013
Epoch 1 step 266: training accuarcy: 0.6905
Epoch 1 step 266: training loss: 1382.8545891421277
Epoch 1 step 267: training accuarcy: 0.6885
Epoch 1 step 267: training loss: 1382.4083080815135
Epoch 1 step 268: training accuarcy: 0.6975
Epoch 1 step 268: training loss: 1382.9286292596848
Epoch 1 step 269: training accuarcy: 0.7055
Epoch 1 step 269: training loss: 1382.8100783364525
Epoch 1 step 270: training accuarcy: 0.713
Epoch 1 step 270: training loss: 1383.6546622408448
Epoch 1 step 271: training accuarcy: 0.707
Epoch 1 step 271: training loss: 1382.8825608359637
Epoch 1 step 272: training accuarcy: 0.705
Epoch 1 step 272: training loss: 1383.230800394471
Epoch 1 step 273: training accuarcy: 0.7010000000000001
Epoch 1 step 273: trainin

...

Epoch 6 step 1741: training accuarcy: 0.6955
Epoch 6 step 1741: training loss: 1382.666794818975
Epoch 6 step 1742: training accuarcy: 0.7000000000000001
Epoch 6 step 1742: training loss: 1382.29850806044
Epoch 6 step 1743: training accuarcy: 0.7045
Epoch 6 step 1743: training loss: 1383.816248352409
Epoch 6 step 1744: training accuarcy: 0.6960000000000001
Epoch 6 step 1744: training loss: 1383.292257358835
Epoch 6 step 1745: training accuarcy: 0.6920000000000001
Epoch 6 step 1745: training loss: 1382.367831435093
Epoch 6 step 1746: training accuarcy: 0.711
Epoch 6 step 1746: training loss: 1383.8754677500765
Epoch 6 step 1747: training accuarcy: 0.6845
Epoch 6 step 1747: training loss: 1382.769616835386
Epoch 6 step 1748: training accuarcy: 0.7030000000000001
Epoch 6 step 1748: training loss: 1382.6848318598868
Epoch 6 step 1749: training accuarcy: 0.6845
Epoch 6 step 1749: training loss: 1383.3193289278927
Epoch 6 step 1750: training accuarcy: 0.6890000000000001
Epoch 6 step 1750: tr

 88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                   | 7/8 [14:03<02:03, 123.02s/it]

Epoch: 7
Epoch 7 step 1841: training loss: 1382.0603360998941
Epoch 7 step 1842: training accuarcy: 0.7085
Epoch 7 step 1842: training loss: 1381.9663085083093
Epoch 7 step 1843: training accuarcy: 0.7030000000000001
Epoch 7 step 1843: training loss: 1382.155464600916
Epoch 7 step 1844: training accuarcy: 0.7030000000000001
Epoch 7 step 1844: training loss: 1381.8843937109814
Epoch 7 step 1845: training accuarcy: 0.713
Epoch 7 step 1845: training loss: 1383.6670187086052
Epoch 7 step 1846: training accuarcy: 0.6920000000000001
Epoch 7 step 1846: training loss: 1381.3106253079434
Epoch 7 step 1847: training accuarcy: 0.715
Epoch 7 step 1847: training loss: 1381.496890954251
Epoch 7 step 1848: training accuarcy: 0.7135
Epoch 7 step 1848: training loss: 1382.0257215977017
Epoch 7 step 1849: training accuarcy: 0.7145
Epoch 7 step 1849: training loss: 1382.4808050050588
Epoch 7 step 1850: training accuarcy: 0.7125
Epoch 7 step 1850: training loss: 1381.6245515092833
Epoch 7 step 1851: train

Epoch 7 step 2005: training accuarcy: 0.677
Epoch 7 step 2005: training loss: 1382.4426488788642
Epoch 7 step 2006: training accuarcy: 0.7115
Epoch 7 step 2006: training loss: 1384.0143155934102
Epoch 7 step 2007: training accuarcy: 0.6895
Epoch 7 step 2007: training loss: 1381.786588647689
Epoch 7 step 2008: training accuarcy: 0.7020000000000001
Epoch 7 step 2008: training loss: 1382.724263304761
Epoch 7 step 2009: training accuarcy: 0.6945
Epoch 7 step 2009: training loss: 1383.3853444019803
Epoch 7 step 2010: training accuarcy: 0.6990000000000001
Epoch 7 step 2010: training loss: 1382.1493924876793
Epoch 7 step 2011: training accuarcy: 0.7115
Epoch 7 step 2011: training loss: 1382.429153467883
Epoch 7 step 2012: training accuarcy: 0.711
Epoch 7 step 2012: training loss: 1381.6580251088676
Epoch 7 step 2013: training accuarcy: 0.707
Epoch 7 step 2013: training loss: 1382.0272257235463
Epoch 7 step 2014: training accuarcy: 0.715
Epoch 7 step 2014: training loss: 1383.463280344703
Epoc

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [16:07<00:00, 123.26s/it]


In [32]:
del prme_model
T.cuda.empty_cache()

### Train Trans FM Model

In [33]:
trans_model = TorchTransFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
trans_model

TorchTransFM()

In [34]:
adam_opt = optim.Adam(trans_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [42]:
trans_learner = FMLearner(trans_model, adam_opt, schedular, db)
trans_learner

In [ ]:
trans_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=trans_loss_callback)

In [36]:
trans_learner.fit(epoch=5,
                  log_dir=get_log_dir('seq_topcoder', 'trans'))

  0%|                                                                                                                                                                 | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 37807.26401193977
Epoch 0 step 1: training accuarcy: 0.4955
Epoch 0 step 1: training loss: 36749.778786841816
Epoch 0 step 2: training accuarcy: 0.504
Epoch 0 step 2: training loss: 35706.60817088014
Epoch 0 step 3: training accuarcy: 0.5
Epoch 0 step 3: training loss: 34674.50367937326
Epoch 0 step 4: training accuarcy: 0.5275
Epoch 0 step 4: training loss: 33686.3987004469
Epoch 0 step 5: training accuarcy: 0.5125
Epoch 0 step 5: training loss: 32706.96820491349
Epoch 0 step 6: training accuarcy: 0.5165
Epoch 0 step 6: training loss: 31751.5109393562
Epoch 0 step 7: training accuarcy: 0.505
Epoch 0 step 7: training loss: 30820.65553823673
Epoch 0 step 8: training accuarcy: 0.524
Epoch 0 step 8: training loss: 29908.863742371064
Epoch 0 step 9: training accuarcy: 0.522
Epoch 0 step 9: training loss: 29012.36624609563
Epoch 0 step 10: training accuarcy: 0.529
Epoch 0 step 10: training loss: 28151.494881656392
Epoch 0 step 11: training accuarcy: 0

Epoch 0 step 172: training accuarcy: 0.7435
Epoch 0 step 172: training loss: 1142.4687222205114
Epoch 0 step 173: training accuarcy: 0.7425
Epoch 0 step 173: training loss: 1148.4498302713594
Epoch 0 step 174: training accuarcy: 0.748
Epoch 0 step 174: training loss: 1126.3964042349307
Epoch 0 step 175: training accuarcy: 0.751
Epoch 0 step 175: training loss: 1175.33577255694
Epoch 0 step 176: training accuarcy: 0.7195
Epoch 0 step 176: training loss: 1150.6492550502567
Epoch 0 step 177: training accuarcy: 0.738
Epoch 0 step 177: training loss: 1154.2970654569183
Epoch 0 step 178: training accuarcy: 0.751
Epoch 0 step 178: training loss: 1132.282888711028
Epoch 0 step 179: training accuarcy: 0.7485
Epoch 0 step 179: training loss: 1129.9585267991922
Epoch 0 step 180: training accuarcy: 0.755
Epoch 0 step 180: training loss: 1151.8311434767393
Epoch 0 step 181: training accuarcy: 0.737
Epoch 0 step 181: training loss: 1144.1199802066885
Epoch 0 step 182: training accuarcy: 0.7335
Epoch

 12%|███████████████████                                                                                                                                     | 1/8 [02:13<15:37, 133.97s/it]

Epoch: 1
Epoch 1 step 263: training loss: 968.1247141387189
Epoch 1 step 264: training accuarcy: 0.8155
Epoch 1 step 264: training loss: 974.9155207398045
Epoch 1 step 265: training accuarcy: 0.799
Epoch 1 step 265: training loss: 968.9799277918569
Epoch 1 step 266: training accuarcy: 0.8115
Epoch 1 step 266: training loss: 969.3743419102765
Epoch 1 step 267: training accuarcy: 0.8095
Epoch 1 step 267: training loss: 975.4692887293817
Epoch 1 step 268: training accuarcy: 0.802
Epoch 1 step 268: training loss: 986.443471727153
Epoch 1 step 269: training accuarcy: 0.801
Epoch 1 step 269: training loss: 965.1668782718472
Epoch 1 step 270: training accuarcy: 0.809
Epoch 1 step 270: training loss: 978.3685398036984
Epoch 1 step 271: training accuarcy: 0.7915
Epoch 1 step 271: training loss: 977.2293235825683
Epoch 1 step 272: training accuarcy: 0.799
Epoch 1 step 272: training loss: 976.9488489259388
Epoch 1 step 273: training accuarcy: 0.798
Epoch 1 step 273: training loss: 968.67655195534

...

Epoch 6 step 1747: training loss: 239.25604234662714
Epoch 6 step 1748: training accuarcy: 0.984
Epoch 6 step 1748: training loss: 250.5090920832568
Epoch 6 step 1749: training accuarcy: 0.9795
Epoch 6 step 1749: training loss: 270.89029412113
Epoch 6 step 1750: training accuarcy: 0.978
Epoch 6 step 1750: training loss: 252.03609499469488
Epoch 6 step 1751: training accuarcy: 0.983
Epoch 6 step 1751: training loss: 260.92424193545634
Epoch 6 step 1752: training accuarcy: 0.978
Epoch 6 step 1752: training loss: 250.57131871772398
Epoch 6 step 1753: training accuarcy: 0.979
Epoch 6 step 1753: training loss: 260.8822594586854
Epoch 6 step 1754: training accuarcy: 0.981
Epoch 6 step 1754: training loss: 277.7113655219941
Epoch 6 step 1755: training accuarcy: 0.97
Epoch 6 step 1755: training loss: 244.93968000144952
Epoch 6 step 1756: training accuarcy: 0.985
Epoch 6 step 1756: training loss: 244.50587167179933
Epoch 6 step 1757: training accuarcy: 0.9825
Epoch 6 step 1757: training loss: 2

 88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                   | 7/8 [15:33<02:12, 132.84s/it]

Epoch: 7
Epoch 7 step 1841: training loss: 207.70664988120637
Epoch 7 step 1842: training accuarcy: 0.987
Epoch 7 step 1842: training loss: 201.9350882288199
Epoch 7 step 1843: training accuarcy: 0.9905
Epoch 7 step 1843: training loss: 200.32081485993274
Epoch 7 step 1844: training accuarcy: 0.9895
Epoch 7 step 1844: training loss: 211.9872521946416
Epoch 7 step 1845: training accuarcy: 0.9875
Epoch 7 step 1845: training loss: 211.92228309816176
Epoch 7 step 1846: training accuarcy: 0.9875
Epoch 7 step 1846: training loss: 201.46405688122888
Epoch 7 step 1847: training accuarcy: 0.99
Epoch 7 step 1847: training loss: 209.99295785035767
Epoch 7 step 1848: training accuarcy: 0.99
Epoch 7 step 1848: training loss: 212.418770928042
Epoch 7 step 1849: training accuarcy: 0.9915
Epoch 7 step 1849: training loss: 198.96235930242898
Epoch 7 step 1850: training accuarcy: 0.989
Epoch 7 step 1850: training loss: 210.20298751367457
Epoch 7 step 1851: training accuarcy: 0.9865
Epoch 7 step 1851: tr

Epoch 7 step 2009: training loss: 189.85619358721257
Epoch 7 step 2010: training accuarcy: 0.989
Epoch 7 step 2010: training loss: 190.03284790052487
Epoch 7 step 2011: training accuarcy: 0.9855
Epoch 7 step 2011: training loss: 188.68278290596368
Epoch 7 step 2012: training accuarcy: 0.985
Epoch 7 step 2012: training loss: 192.35876979405282
Epoch 7 step 2013: training accuarcy: 0.99
Epoch 7 step 2013: training loss: 194.51239370097494
Epoch 7 step 2014: training accuarcy: 0.989
Epoch 7 step 2014: training loss: 198.96255752038684
Epoch 7 step 2015: training accuarcy: 0.991
Epoch 7 step 2015: training loss: 195.3538354390031
Epoch 7 step 2016: training accuarcy: 0.9875
Epoch 7 step 2016: training loss: 205.26478259370407
Epoch 7 step 2017: training accuarcy: 0.9865
Epoch 7 step 2017: training loss: 198.26959940048633
Epoch 7 step 2018: training accuarcy: 0.986
Epoch 7 step 2018: training loss: 199.2528931640351
Epoch 7 step 2019: training accuarcy: 0.9865
Epoch 7 step 2019: training l

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [17:43<00:00, 132.13s/it]


In [44]:
del trans_model
T.cuda.empty_cache()